In [ ]:
% run 0-utils.ipynb

In [ ]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import torch.nn.functional as F
from skorch.net import NeuralNetClassifier
from sklearn.preprocessing import StandardScaler
from gcnn.utils import NNplusplus
from gcnn.nets import *

In [ ]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [ ]:
from gcnn.datasets import load_icebergs

In [ ]:
icebergs = load_icebergs('train')

In [ ]:
for _, i in icebergs.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [ ]:
y = icebergs.is_iceberg.values.astype(np.float32)
x = (
    np.stack(
        [
            scaler_1.transform(np.stack(icebergs.band_1)),
            scaler_2.transform(np.stack(icebergs.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).astype(np.float32)

In [ ]:
x.dtype, y.dtype

In [ ]:
type(x), type(y)

In [ ]:
y.shape, x.shape

In [ ]:
insize1 = 2 
outsize1 = 32 
outsize2 = 32
outsize3 = 32 
outsize4 = 32 

class Net(nn.Module):
    def __init__(self,outsize1=32,outsize2=32):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(insize1, outsize1, kernel_size=5, stride=1, padding=2, groups=2),
            nn.BatchNorm2d(outsize1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(outsize1, outsize2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(outsize2, outsize3, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(outsize3, outsize4, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = NNplusplus(
    Net,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, #regularization
    max_epochs=250,
    batch_size=25,
    lr=0.000001,
    use_cuda = True
)

In [ ]:
x.dtype, y.dtype

In [ ]:
model.fit(x,y)

In [ ]:
%%capture output4
from sklearn.model_selection import GridSearchCV


params = {
    'lr': [0.000001, 0.000002,0.000004,0.000008],
    'max_epochs': [50,100,150, 200,250],
}
gs = GridSearchCV(model, params, refit=False, cv=3,n_jobs=1)

In [ ]:
gs.fit(x, y)

In [ ]:
gs

In [ ]:
gs